In [1]:
import tensorflow as tf
tf.__version__

'1.1.0'

### constant


In [39]:
sess = tf.InteractiveSession()
a=tf.zeros([2,3],tf.int32)
b=tf.ones([2,3],tf.int32)
c=tf.constant([[3., 3.]])
print(sess.run(a))
print(sess.run(b))    
print(sess.run(c)) 


[[0 0 0]
 [0 0 0]]
[[1 1 1]
 [1 1 1]]
[[ 3.  3.]]


### 随机数

In [41]:
tf.random_normal(shape=[2,3], mean=0.0, stddev=1.0)
tf.random_uniform(shape=[2,3], minval=0.0, maxval=1.0)

<tf.Tensor 'random_uniform:0' shape=(2, 3) dtype=float32>

In [23]:
help(tf.random_normal)

Help on function random_normal in module tensorflow.python.ops.random_ops:

random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
    Outputs random values from a normal distribution.
    
    Args:
      shape: A 1-D integer Tensor or Python array. The shape of the output tensor.
      mean: A 0-D Tensor or Python value of type `dtype`. The mean of the normal
        distribution.
      stddev: A 0-D Tensor or Python value of type `dtype`. The standard deviation
        of the normal distribution.
      dtype: The type of the output.
      seed: A Python integer. Used to create a random seed for the distribution.
        See
        [`set_random_seed`](../../api_docs/python/constant_op.md#set_random_seed)
        for behavior.
      name: A name for the operation (optional).
    
    Returns:
      A tensor of the specified shape filled with random normal values.




### 损失函数


In [32]:
#交叉熵
#二分类
cross_entroy=tf.nn.sigmoid_cross_entropy_with_logits(y,y_)
#多分类
cross_entroy=tf.nn.softmax_cross_entropy_with_logits(y,y_)
#自定义
cross_entroy=-tf.reduce_mean(y_*tf.log(tf.clip_by_value(y,le-10,1.0)))
loss=tf.reuce_sum(tf.select(tf.greater(v1,v2),(v1-v2)*1,(v2-v1)*10))

NameError: name 'y_' is not defined

In [ ]:
#L1 , L2 ,lambda为正则化项系数
l2=tf.contrib.layers.l1_regularizer(lambda)(w)
l2=tf.contrib.layers.l2_regularizer(lambda)(w)
loss=tf.reduce_mean(tf.reduce_mean(tf.square(y-y_)+ l2)) 


### 优化算法

In [42]:
tf.train.GradientDescentOptimizer(leanring_rate).minimize(loss)
tf.train.AdamOptimizer(leanring_rate).minimize(loss)

NameError: name 'leanring_rate' is not defined

### 衰减率


In [ ]:
tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay_rate, staircase=False, name=None)

In [43]:
help(tf.train.ExponentialMovingAverage)

Help on class ExponentialMovingAverage in module tensorflow.python.training.moving_averages:

class ExponentialMovingAverage(builtins.object)
 |  Maintains moving averages of variables by employing an exponential decay.
 |  
 |  When training a model, it is often beneficial to maintain moving averages of
 |  the trained parameters.  Evaluations that use averaged parameters sometimes
 |  produce significantly better results than the final trained values.
 |  
 |  The `apply()` method adds shadow copies of trained variables and add ops that
 |  maintain a moving average of the trained variables in their shadow copies.
 |  It is used when building the training model.  The ops that maintain moving
 |  averages are typically run after each training step.
 |  The `average()` and `average_name()` methods give access to the shadow
 |  variables and their names.  They are useful when building an evaluation
 |  model, or when restoring a model from a checkpoint file.  They help use the
 |  movin

In [115]:
#生成一个指数平滑对象
steps=tf.Variable(0)
ema=tf.train.ExponentialMovingAverage(decay=0.99,num_updates=steps)
#apply
v1=tf.Variable(0,dtype=tf.float32) 
maintain_averages_op =ema.apply([v1])
# 初始化向量
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op) 
    # 通过 ema.average(v1)获取平滑之后的变量的值
    print(sess.run([v1,ema.average(v1)])) #[0.0, 0.0]
    
    # 将v1值更新到5
    sess.run(tf.assign(v1,5)) 
    sess.run(maintain_averages_op)   
    print(sess.run([v1,ema.average(v1)])) #[5.0, 4.5]
    
    # step =1000
    sess.run(tf.assign(v1,10)) 
    sess.run(tf.assign(steps,1000))   # decay = 0.99
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)])) #[10.0, 4.5549998]
    # 0.99 * 4.5 + 0.01 * 10=4.55  # 0.99 * 4.5 , 4.5是上次shadow variable的值


[0.0, 0.0]
[5.0, 4.5]
[10.0, 9.4499998]


In [114]:
v2=tf.Variable(5 ,dtype=tf.float32)
v3=tf.Variable(10,dtype=tf.float32)
ema=tf.train.ExponentialMovingAverage(decay=0.9,num_updates=steps)
maintain_averages_op =ema.apply([v2,v3])
# 初始化向量
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op) 
    # 通过 ema.average(v1)获取平滑之后的变量的值
    print(sess.run([v2,ema.average(v2)])) #[0.0, 0.0]
    print(sess.run([v3,ema.average(v3)]))
    
    # 通过 ema.average(v1)获取平滑之后的变量的值
    v2=tf.assign(v2 ,10)
    print(sess.run([v2,ema.average(v2)])) #[0.0, 0.0] 
    


[5.0, 5.0]
[10.0, 10.0]
[5.0, 5.0]
[10.0, 10.0]


In [7]:
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(0.1, global_step, 10, 2, staircase=False)
init_op = tf.global_variables_initializer()
with tf.Session() as sess:  
    sess.run(init_op)
    for i in range(10):
        print(sess.run(learning_rate))  
        print(sess.run(global_step)  )

0.1
0
0.1
0
0.1
0
0.1
0
0.1
0
0.1
0
0.1
0
0.1
0
0.1
0
0.1
0


In [40]:
import numpy as np
x = tf.placeholder(tf.float32, shape=[None, 1], name='x')  
y = tf.placeholder(tf.float32, shape=[None, 1], name='y')  
w = tf.Variable(tf.constant(0.0))  
  
global_step = tf.Variable(0, trainable=False)  
learning_rate = tf.train.exponential_decay(0.1, global_step, 10, 2, staircase=False)  
loss = tf.pow(w*x-y, 2)  
  
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  
  
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())  
    for i in range(10):  
        sess.run(train_step, feed_dict={x:np.linspace(1,2,10).reshape([10,1]),  
            y:np.linspace(1,2,10).reshape([10,1])})  
        print(sess.run(learning_rate)  )
        print(sess.run(global_step) )

0.107177
1
0.11487
2
0.123114
3
0.131951
4
0.141421
5
0.151572
6
0.16245
7
0.17411
8
0.186607
9
0.2
10


In [39]:
global_steps = tf.Variable(0, trainable=False)  
learning_rate = tf.train.exponential_decay(0.1, global_steps, 10, 2, staircase=False)  

variable_averages = tf.train.ExponentialMovingAverage( 0.99)
variables_averages_op = variable_averages.apply(global_steps)

with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer()) 
    sess.run([learning_rate,variables_averages_op])
    for i in range(10):     
        print(sess.run(learning_rate)  )
        print(sess.run(global_steps) )

TypeError: 'Variable' object is not iterable.

In [37]:
a=tf.trainable_variables()[1].name
with tf.Session() as sess:
    print(sess.run(tf.get_variable(a)))

ValueError: Shape of a new variable (Variable_7:0) must be fully defined, but instead was <unknown>.

In [119]:
#保存
saver=tf.train.Saver()
with tf.Session() as sess:
    saver.save(sess,"path/model/model.ckpt")    


ValueError: Parent directory of /model/model.ckpt doesn't exist, can't save.

In [120]:
# load
saver = tf.train.Saver()
# 若只加载图
saver2 = tf.train.import_meta_graph("path/model/model.ckpt/model.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess,"/model/model.ckpt") 
    saver2.restore(sess,"/model/model.ckpt")
    
    

OSError: File path/model/model.ckpt/model.ckpt.meta does not exist.

In [130]:
q=tf.FIFOQueue(2,"int32")
init=q.enqueue_many(([0,10],))
#
x=q.dequeue()
y=x+1
q_inc=q.enqueue([y])

with tf.Session() as sess:
    init.run()
    for i in range(5):
        v,i=sess.run([x,q_inc])
        print(v)

0
10
1
11
2


In [54]:
with tf.variable_scope("test"):
    v1 = tf.get_variable(name='v1', shape=[1], initializer=tf.constant_initializer(1))   
    v2 = tf.get_variable(name='v2', shape=[1], initializer=tf.constant_initializer(2)) 

tf.add_to_collection('a', v1)
tf.add_to_collection('a', v2) 
c = tf.add_n(tf.get_collection('a') ) 
init = tf.global_variables_initializer()
with tf.Session() as sess: 
    sess.run(init)  
    print(sess.run(c))

[ 3.]


In [44]:
a = tf.constant(2) 
print(a)

Tensor("Const_2:0", shape=(), dtype=int32)


In [30]:
v=tf.Variable(1,dtype=tf.int32)
print(v.name)

Variable_3:0


In [52]:
with tf.variable_scope("test",reuse=True):
    v=tf.get_variable("v",shape=[1],initializer=tf.constant_initializer(1))
#update
with tf.Session() as sess:
    sess.run(v.initializer)
    print(sess.run(v))
    #update
    sess.run(v.assign([2]))
    print(sess.run(v))

[ 1.]
[ 2.]


In [27]:
with tf.Session() as sess:
    sess.run(v1.initializer)
    print(sess.run(v1))
    # update
    sess.run(v1)

[ 0.]
